In [2]:
import os
from numba import njit
import pandas as pd
import sys
import sqlalchemy
from pandas.tseries.offsets import BDay
from datetime import datetime

import sys
sys.path.insert(0, r'C:\Scripts\modules\database')
sys.path.insert(0, r'C:\Scripts\modules\parameters')

from connection import Connect
from bases import Bases
from parametros import Parametros

In [3]:
conn = Connect.connect_techdb()
base = Connect.import_table(conn, "base_btg")
base['Conta'] = base['Conta'].astype('str')
nomes_clientes = Connect.import_table(conn, "nomes_clientes")
conn.close()

In [4]:
# %%
diretorio_raiz = r"C:\Scripts\relatórios"
pastas_dias = []

# 1. Coleta as pastas diretamente na raiz
for item in os.listdir(diretorio_raiz):
    caminho_completo = os.path.join(diretorio_raiz, item)
    
    # Verifica se é uma pasta e se o nome tem o formato de data (DD-MM-YYYY)
    if os.path.isdir(caminho_completo):
        try:
            # Valida se o nome da pasta é uma data válida
            datetime.strptime(item, "%d-%m-%Y")
            pastas_dias.append({"dia": item})
        except ValueError:
            # Ignora pastas que não são datas 
            continue

# 2. Ordena as pastas cronologicamente
pastas_dias.sort(key=lambda x: datetime.strptime(x["dia"], "%d-%m-%Y"))

# Debug para você conferir as últimas pastas encontradas
if pastas_dias:
    print(f"Sucesso! Total de pastas encontradas: {len(pastas_dias)}")
    print(f"Primeira pasta: {pastas_dias[0]['dia']}")
    print(f"Última pasta: {pastas_dias[-1]['dia']}")
else:
    print("Nenhuma pasta de data encontrada no diretório raiz.")

Sucesso! Total de pastas encontradas: 31
Primeira pasta: 27-11-2025
Última pasta: 29-01-2026


In [5]:
conn = Connect.connect_techdb()
entradas_consolidado = Connect.import_table(conn, "Entradas_e_saidas_consolidado")
conn.close()

In [6]:
# Identifica a última data processada no banco
max_data_banco = pd.to_datetime(entradas_consolidado['Mês de entrada/saída'].max(), format='%Y-%m-%d')
print(f"Data mais recente no banco: {max_data_banco.date()}")

Data mais recente no banco: 2026-01-28


In [7]:
# OBS: Se a lista ficar vazia ou tiver apenas 1 item, o processo para aqui.
pastas_para_processar = [
    p for p in pastas_dias 
    if pd.to_datetime(p['dia'], format='%d-%m-%Y') > max_data_banco
]

pasta_base_anterior = next((p for p in pastas_dias if pd.to_datetime(p['dia'], format='%d-%m-%Y') == max_data_banco), None)

if pasta_base_anterior and pastas_para_processar:
    # Adiciona a data base no início da lista para permitir o zip correto
    pastas_para_processar.insert(0, pasta_base_anterior)
elif not pasta_base_anterior and pastas_para_processar:
    # Se não achou a pasta do dia anterior (talvez tenha sido deletada), avisa mas tenta seguir
    print("Aviso: A pasta da data base (última do banco) não foi encontrada no diretório. O primeiro dia novo pode ser pulado.")

In [8]:
lista_entrada_saida = []
diretorio_base = r"C:\Scripts\relatórios"

if len(pastas_para_processar) < 2:
    print("Não há dias suficientes para comparação (Banco já está atualizado).")

else:
    print(f"Iniciando processamento de {len(pastas_para_processar)-1} novos dias...")

    # O zip garante que comparamos o dia atual (presente) com o dia anterior (passado)
    for passado, presente in zip(pastas_para_processar[:-1], pastas_para_processar[1:]):
        
        dia_passado_str = passado['dia']
        dia_presente_str = presente['dia']
        
        try:
            # --- BUSCA BASE ANTERIOR ---
            caminho_dia_passado = os.path.join(diretorio_base, dia_passado_str)
            subpasta_passado = os.listdir(caminho_dia_passado)[0]
            arquivo_passado = os.path.join(caminho_dia_passado, subpasta_passado, "base_btg.xlsx")
            
            try:
                base_anterior = pd.read_excel(arquivo_passado, header=2)
                if 'Conta' not in base_anterior.columns:
                    base_anterior = pd.read_excel(arquivo_passado)
            except:
                base_anterior = pd.read_excel(arquivo_passado)
            
            base_anterior['Conta'] = base_anterior['Conta'].astype(str)

            # --- BUSCA BASE ATUAL ---
            caminho_dia_presente = os.path.join(diretorio_base, dia_presente_str)
            subpasta_presente = os.listdir(caminho_dia_presente)[0]
            arquivo_atual = os.path.join(caminho_dia_presente, subpasta_presente, "base_btg.xlsx")
            
            try:
                base_atual = pd.read_excel(arquivo_atual, header=2)
                if 'Conta' not in base_atual.columns:
                    base_atual = pd.read_excel(arquivo_atual)
            except:
                base_atual = pd.read_excel(arquivo_atual)
                
            base_atual['Conta'] = base_atual['Conta'].astype(str)

            # --- LÓGICA DE ENTRADA E SAÍDA ---
            # Clientes que estavam na anterior e NÃO estão na atual (SAÍRAM)
            # Nota: O Mês de saída deve ser o do dia PRESENTE (data do evento), mas seu código original usava passado. 
            # Mantive a lógica original, mas verifique se não prefere `dia_presente_str`.
            clientes_sairam = base_anterior[~base_anterior['Conta'].isin(base_atual['Conta'])].copy()
            clientes_sairam['Situação'] = "Saiu"
            clientes_sairam['Mês de entrada/saída'] = dia_presente_str # Ajustado para data do evento

            # Clientes que NÃO estavam na anterior e estão na atual (ENTRARAM)
            clientes_entraram = base_atual[~base_atual['Conta'].isin(base_anterior['Conta'])].copy()
            clientes_entraram['Situação'] = "Entrou"
            clientes_entraram['Mês de entrada/saída'] = dia_presente_str # Ajustado para data do evento
            
            if 'Faixa Cliente' in clientes_entraram.columns:
                clientes_entraram.drop("Faixa Cliente", axis=1, inplace=True)
            
            # Consolida
            entradas_saidas = pd.concat([clientes_sairam, clientes_entraram], axis=0)
            
            colunas_alvo = ['Conta', 'Assessor', "PL Total", "PL Declarado", "Faixa Cliente", 'Data Vínculo', 'Situação', "Mês de entrada/saída"]
            colunas_existentes = [c for c in colunas_alvo if c in entradas_saidas.columns]
            
            entradas_saidas = entradas_saidas[colunas_existentes]
            
            entradas_saidas = entradas_saidas[~entradas_saidas['Conta'].isin(['1983816', '1106619'])]
            entradas_saidas = pd.merge(entradas_saidas, nomes_clientes, on='Conta', how='left')

            if not entradas_saidas.empty:
                lista_entrada_saida.append(entradas_saidas)
                print(f"[OK] Comparado: {dia_passado_str} -> {dia_presente_str} (Gerou {len(entradas_saidas)} linhas)")
            else:
                print(f"[OK] Comparado: {dia_passado_str} -> {dia_presente_str} (Nenhuma alteração encontrada)")

        except Exception as e:
            print(f"[ERRO] Falha ao comparar {dia_passado_str} com {dia_presente_str}: {e}")

# --- CONSOLIDAÇÃO FINAL COM PROTEÇÃO ---
if lista_entrada_saida:
    entradas_saidas_consolidado = pd.concat(lista_entrada_saida, ignore_index=True)
    table = entradas_saidas_consolidado.copy()
    print(f"{len(table)} novas movimentações identificadas.")
else:
    print("Lista Vazia.")
    # Cria um DataFrame vazio com as colunas esperadas para não quebrar códigos futuros
    table = pd.DataFrame(columns=['Conta', 'Situação', 'Mês de entrada/saída'])

Iniciando processamento de 1 novos dias...
[OK] Comparado: 28-01-2026 -> 29-01-2026 (Gerou 7 linhas)
7 novas movimentações identificadas.


In [9]:
table = pd.concat(lista_entrada_saida)

In [10]:
table

,Conta,Assessor,PL Total,PL Declarado,Faixa Cliente,Situação,Mês de entrada/saída,Nome
0,8630055,CAIC ZEM GOMES,0.00,26700000.0,Ate 50K,Saiu,29-01-2026,Camila
1,16458051,Renan Bento da Silva,NaN,650000.0,NaN,Entrou,29-01-2026,Fernanda
2,17593583,CAIC ZEM GOMES,515.00,1420000.0,NaN,Entrou,29-01-2026,Davi
3,18774067,Paulo Roberto Faria Silva,2030285.94,10000000.0,NaN,Entrou,29-01-2026,Dino
4,21236666,Paulo Roberto Faria Silva,NaN,1000000.0,NaN,Entrou,29-01-2026,Formenton
5,21246129,CAIC ZEM GOMES,NaN,4000000.0,NaN,Entrou,29-01-2026,Marineia
6,21312293,Rodrigo de Mello DElia,NaN,500000.0,NaN,Entrou,29-01-2026,Arion


In [11]:
table = table.drop_duplicates(subset=['Conta'], keep='last')
table['Assessor'] = [name.upper() for name in table['Assessor']]

In [12]:
### Ajuste no nome do Rodrigo
table.loc[table['Assessor'] == "RODRIGO DE MELLO D?ELIA", "Assessor"] = "RODRIGO DE MELLO D’ELIA"
table.loc[table['Assessor'] == "RODRIGO DE MELLO DELIA", "Assessor"] = "RODRIGO DE MELLO D’ELIA"

In [13]:
### Ajuste manual josé
table.loc[table['Conta'] == '590732', 'Assessor'] = 'JOSE AUGUSTO ALVES DE PAULA FILHO'
table.loc[table['Conta'] == '299305', 'Assessor'] = "JOSE AUGUSTO ALVES DE PAULA FILHO"

In [14]:
table['Mês de entrada/saída'] = pd.to_datetime(table['Mês de entrada/saída'], format = "%d-%m-%Y")

In [15]:
conn = Connect.connect_techdb()
table.to_sql('Entradas_e_saidas_consolidado', 
             con=conn, 
             index=False, 
             if_exists='append', 
             dtype={'Conta':sqlalchemy.VARCHAR(255)},
             schema="principal")
conn.close()

In [16]:
conn = Connect.connect_techdb()
migracoes_btg = Connect.import_table(conn, "migracoes_btg")
cap_hist = Connect.import_table(conn, "captacao_historico")
conn.close()

In [17]:
table = table[table['Mês de entrada/saída'] > migracoes_btg['DATA'].max()]

In [18]:
contas_remover = []
for conta in table['Conta']:
    contas = cap_hist[(cap_hist['CONTA'] == conta) & (cap_hist['DATA'] == '{}'.format(table[table['Conta'] == conta]['Mês de entrada/saída']))]
    if len(contas) > 0:
        contas_remover.append(contas)
try:
    contas_remover = pd.concat(contas_remover)
    table = table[~table['Conta'].isin(contas_remover['CONTA'])]
    print(contas_remover)
except:
    pass

In [19]:
adicionar_em_captacao = table[['Mês de entrada/saída', 'Conta', 'Assessor', 'PL Total', 'Situação']]
adicionar_em_captacao = adicionar_em_captacao[adicionar_em_captacao['Situação'] == "Entrou"]
adicionar_em_captacao.drop("Situação", axis=1, inplace=True)
adicionar_em_captacao.rename(columns={
    'Mês de entrada/saída' : "DATA",
    "Conta" : "CONTA",
    "PL Total" : "CAPTAÇÃO"
}, inplace=True)
adicionar_em_captacao['Considerar'] = "SIM"
adicionar_em_captacao.dropna(subset='CAPTAÇÃO', inplace=True)
conn = Connect.connect_techdb()
adicionar_em_captacao.to_sql("migracoes_btg", 
                             con=conn, 
                             index=False, 
                             if_exists='append', 
                             dtype={'CONTA':sqlalchemy.VARCHAR(255)},
                             schema="principal")
conn.close()

In [20]:
adicionar_em_captacao

,DATA,CONTA,Assessor,CAPTAÇÃO,Considerar
2,2026-01-29,17593583,CAIC ZEM GOMES,515.00,SIM
3,2026-01-29,18774067,PAULO ROBERTO FARIA SILVA,2030285.94,SIM
